In [ ]:
%load_ext tensorboard

#
#
# start Tensorboard with CMD-Shft-P Python:Launch TensorBoard

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.image import AxesImage, NonUniformImage
import numpy as np
from datetime import datetime, timedelta
import math
from scipy.interpolate import interp1d
from PIL import Image
import cv2
import shutil
import random
import uuid
from prettytable import PrettyTable

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from PIL import Image
from sklearn.utils import shuffle

import tensorflow as tf

sys.path.append(os.path.join(os.getcwd(),'..'))
from lib import find_nearest_index, FigureSize




In [ ]:
from datetime import datetime

In [ ]:
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)

In [ ]:
NUM_OUTPUTS = 26 # no. of peaks
NUM_BATCHES = 32
NUM_EPOCHS = 100
IMAGE_SIZE = 32
#NUM_TRAIN_LABELS = 2600 # use outpuf of load_images()

TRAIN_DATA_PATH= os.path.join(os.getcwd(), '..','data','train')
TEST_DATA_PATH = os.path.join(os.getcwd(), '..','data','test')
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
MODEL_PATH = os.path.join(os.getcwd(), '..','data','model'+timestamp+'.h5')

TESTDATA_TRAINDATA_RATIO = 20./80.

In [ ]:
NEON_REFERENCE_FILE = os.path.join(os.getcwd(),'..','data','ref','NIST','Ne','neon-exported.csv')
WAVELENGTHS_MIN, WAVELENGTHS_MAX = 4000, 9000
WINDOW = 512
STEPSIZE_MIN, STEPSIZE_MAX, STEPSIZE_N = 0.5, 1.5, 2000


In [ ]:
RANDOM_NUMBER = math.pow(10,int(math.log10(STEPSIZE_N *NUM_OUTPUTS))+1)
print (RANDOM_NUMBER)

In [ ]:
MAX_SSIM = 0.6
PERCENT = 1.0
DROPOUT = 0.5

In [ ]:
def scale(min_scale, max_scale):
    return random.random()*(max_scale - min_scale)+0.95

In [ ]:
def gauss():
    return random.gauss(1.0,0.3)

In [ ]:
def build_model():
        
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Reshape((IMAGE_SIZE,IMAGE_SIZE,1),input_shape=(IMAGE_SIZE,IMAGE_SIZE,1)))
    # 
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) 
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) 
    # prediction part        
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.Dropout(DROPOUT))
    # model.add(tf.keras.layers.Dense(NUM_TRAIN_LABELS, activation='softmax')) 
    model.add(tf.keras.layers.Dense(NUM_OUTPUTS, activation='softmax')) 

    return model

In [ ]:
def load_image(image_path):
    img =cv2.imread(image_path)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.reshape([IMAGE_SIZE, IMAGE_SIZE,1])
    return img

In [ ]:
def load_images(path):
    train_images = []
    train_labels = []

    dir_index = 0
    for directory in sorted(os.listdir(path)):
        if(directory.startswith('.') == False):

            
            for filename in sorted(os.listdir(os.path.join(path, directory))):

                if(filename.startswith('.') == False):
                        
                        image_path = os.path.join(path, directory,filename    )
                        img =cv2.imread(image_path)
                        img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        img = img.reshape([IMAGE_SIZE, IMAGE_SIZE,1])
                        train_images.append(img)
                train_labels.append(dir_index)
            dir_index = dir_index + 1

    print(len(train_images),train_labels)
    return np.array(train_images)/255, np.array(tf.keras.utils.to_categorical(train_labels,NUM_OUTPUTS))#len(train_labels)))


In [ ]:
def count_probabilities(probs):
    count_0 = (0.9 <= probs).sum()
    count_1 = ((0.1 < probs) & (probs < 0.9)).sum()
    count_2 = ((0.01 <= probs) & (probs < 0.1)).sum()
    count_e10 = ((1.0e-10 <= probs) & (probs < 0.01 )).sum()
    count_e20 = ((1.0e-20 <= probs) & (probs < 1e-10 )).sum()
    count_e30 = ((1.0e-30 <= probs) & (probs < 1e-20 )).sum()
    count_e40 = ((1.0e-40 <= probs) & (probs < 1e-30 )).sum()
    counts = count_0 + count_1 + count_2 + count_e10 + count_e20 + count_e30 + count_e40
    return counts

In [ ]:
# Test auf Dataset
def test_dataset(model, labels_decoded, path, image_size=IMAGE_SIZE, resultfile=None):
    correct_matches = 0
    wrong_matches = 0
    result_table = PrettyTable()
    result_table.field_names = ["Datei ", "Ist", "Dekodiert", "Match?"]

    for filename in sorted(os.listdir(path)):

        if(filename.startswith('.') == False):

            current_wavelength = filename[0:7]
            #print (current_wavelength)
            image_path = os.path.join(path,filename)
                
            test_image = load_image(image_path)
            predictions = model.predict(test_image.reshape((1,IMAGE_SIZE,IMAGE_SIZE,1)), verbose=0)
            counts = count_probabilities(predictions)
                
            index_max_predictions = np.argmax(predictions)
            #print(predictions)
            #print('index_max_predictions:',index_max_predictions, current_wavelength, labels_decoded[index_max_predictions])
            decode_wavelength = labels_decoded[index_max_predictions]
            # Passt oder nicht?
            if str.upper(current_wavelength) == str.upper(decode_wavelength):
                mark = "✅ {:02d}".format(counts)
                correct_matches += 1 
            else:
                mark = "❌ {:02d}".format(counts)
                wrong_matches += 1
            result_table.add_row([image_path, current_wavelength, decode_wavelength, mark ])


    #print(result_table)
    if resultfile is None:
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        resultfile = "result_"+timestamp+".txt"
    with open(resultfile,"w") as result_f:
        result_f.write(str(result_table))
    return correct_matches, wrong_matches

In [ ]:
neon_reference_file = os.path.join(NEON_REFERENCE_FILE)
positions = {'wavelength':2, 'intensity':6, 'selector':1}
selector = '1'
intensity_limit = 1.0
neon_wavelengths = []
neon_intensities = []

with open(neon_reference_file,'r') as neon_f:
    for line in neon_f:
        if line.startswith('#'):
            pass
        else:
            tokens =  line.split(';')
            #print (tokens)
            try:
                if selector in tokens[positions['selector']] :
                    
                    neon_wavelength, neon_intensity = float(tokens[positions['wavelength']]), float(tokens[positions['intensity']])
                    
                    if neon_intensity > intensity_limit :
                        
                        neon_wavelengths.append(neon_wavelength)
                        neon_intensities.append(neon_intensity)
                        
                        print ("{:8.3f} {:10.0f}".format(neon_wavelength, neon_intensity, ))
                    
            except ValueError:
                pass

print (len(neon_wavelengths), len(neon_intensities))
#

In [ ]:
neon_reference_file = 'linetable-NE.csv'
positions = {'wavelength':0, 'intensity':2}
neon_wavelengths = []
neon_intensities = []

with open(neon_reference_file,'r') as neon_f:
    for line in neon_f:
        if line.startswith('#'):
            pass
        else:
            tokens =  line.split(';')
            print (tokens)
            try:
                neon_wavelength, neon_intensity = float(tokens[positions['wavelength']]), float(tokens[positions['intensity']])
                neon_wavelengths.append(neon_wavelength)
                neon_intensities.append(neon_intensity)
                
                print ("{:8.3f} {:10.0f}".format(neon_wavelength, neon_intensity, ))
                
                        
            except ValueError:
                pass

print (len(neon_wavelengths), len(neon_intensities))
#
# print (neon_wavelengths)

In [ ]:
plt.rcParams['figure.figsize'] = FigureSize.NARROW

In [ ]:
wavelengths = np.array(range(WAVELENGTHS_MIN,WAVELENGTHS_MAX,1))*1.0
intensities = wavelengths * 0.0
sigma = 4.0
k = -2*sigma*sigma
r = math.sqrt(2*math.pi*sigma*sigma)

for neon_w, neon_i, in zip(neon_wavelengths, neon_intensities):

    _e = (wavelengths - neon_w)*(wavelengths- neon_w) / k
    intensities = intensities + np.exp(_e) * neon_i
        

xlim = [wavelengths[1], wavelengths[-2]]
_i1 = find_nearest_index(wavelengths,xlim[0])
_i2 = find_nearest_index(wavelengths,xlim[1])
#print (_i1[0], _i2[0])
max_i = intensities[_i1:_i2].max()
normalized_intensities = intensities / max_i
fig, ax = plt.subplots()

plt.plot(wavelengths, normalized_intensities)
for _nw, _ni in zip(neon_wavelengths, neon_intensities):
    plt.text(_nw,_ni/max_i,"{:6.1f}".format(_nw), rotation=90, horizontalalignment='center')
plt.xlim(xlim)
plt.ylim(0,1.1)
plt.show()

In [ ]:
wavelengths = np.array(range(WAVELENGTHS_MIN,WAVELENGTHS_MAX,1))*1.0
pixels = np.array(range(0,len(wavelengths)))
intensities = wavelengths * 0.0
sigma = 4.0
k = -2*sigma*sigma
r = math.sqrt(2*math.pi*sigma*sigma)

for neon_w, neon_i, in zip(neon_wavelengths, neon_intensities):

    _e = (wavelengths - neon_w)*(wavelengths- neon_w) / k
    intensities = intensities + np.exp(_e) * neon_i

xlim = [wavelengths[1], wavelengths[-2]]
_i1 = find_nearest_index(wavelengths,xlim[0])
_i2 = find_nearest_index(wavelengths,xlim[1])
max_i = intensities[_i1:_i2].max()
normalized_intensities = intensities / max_i

ny = WINDOW
nx = len(normalized_intensities)

twod = np.zeros((ny, nx))
for i in range(ny):
    twod[i] = normalized_intensities*-1+1.0

ylim = [0,ny]

n_cols = 5
n_rows = math.ceil(len(neon_wavelengths)/n_cols)

window = WINDOW
window_h = int(window/2)
image_size = [IMAGE_SIZE,IMAGE_SIZE]

plt.rcParams['figure.figsize'] = FigureSize.LARGE
fig, axes = plt.subplots(n_rows, n_cols,sharex=False, sharey=False)

i_row = 0
i_col = 0

for index in range(0,len(neon_wavelengths)):
    neon_w = neon_wavelengths[index]
    
    _xlim = [
            find_nearest_index(wavelengths,neon_w)-window_h,
            find_nearest_index(wavelengths,neon_w)+window_h,
    ]
    _ylim = [0,ny]
    _twod = twod[_ylim[0]:_ylim[1], _xlim[0]:_xlim[1]]
    wavelength_text = str(int(neon_w*100)/100)
    print(index, neon_w, wavelength_text, _xlim)
    

    if n_cols == 1:
        axes[i_row].imshow(_twod, cmap='gray')
        axes[i_row].set_title(wavelength_text)
        axes[i_row].set_ylim(_ylim)
        axes[i_row].set_xlim(0,_xlim[1]-_xlim[0])
        print (i_row, wavelength_text, _xlim)
        i_row += 1
    else:
        axes[i_row, i_col].imshow(_twod, cmap='gray')
        axes[i_row, i_col].set_title(wavelength_text)
        axes[i_row, i_col].set_ylim(_ylim)
        axes[i_row, i_col].set_xlim(0,_xlim[1]-_xlim[0])

        i_col += 1
        if i_col >= n_cols:
            i_col = 0
            i_row +=1
plt.show()

In [ ]:
for neon_w in neon_wavelengths:
    wavelength_text = str(int(neon_w*100)/100)
    p = os.path.join(TRAIN_DATA_PATH, wavelength_text)
    try:
        shutil.rmtree(p, ignore_errors=True)
        
    except FileNotFoundError:
        print ("skipping "+p)
    os.mkdir(p)

In [ ]:
sigma = 4.0
k = -2*sigma*sigma
r = math.sqrt(2*math.pi*sigma*sigma)

xlim = [wavelengths[1], wavelengths[-2]]
ny = WINDOW
nx = len(normalized_intensities)
ylim = [0,ny]


window = WINDOW
window_h = int(window/2)
image_size = [IMAGE_SIZE,IMAGE_SIZE]

filename_counter = [0] * len (neon_wavelengths)

min_stepsize = STEPSIZE_MIN
max_stepsize = STEPSIZE_MAX
n_steps = STEPSIZE_N
d_step = (max_stepsize - min_stepsize)/n_steps

stepsizes = np.array(range(n_steps))*d_step+min_stepsize
#print (stepsizes)
nbins = WAVELENGTHS_MAX - WAVELENGTHS_MIN
for stepsize in stepsizes:
    wavelengths = np.array(range(nbins))*stepsize+WAVELENGTHS_MIN #   np.array(range(4000,9000,stepsize))*1.0
    
    intensities = np.zeros(nbins)

    for neon_w, neon_i, in zip(neon_wavelengths, neon_intensities):
        _neon_w = neon_w+random.gauss(0,1.5)
        _e = (wavelengths - _neon_w)*(wavelengths- _neon_w) / k
        intensities = intensities + np.exp(_e) * neon_i*gauss() # scale(0.9,1.0)

    max_i = intensities[1:-2].max()
    normalized_intensities = intensities / max_i

    twod = np.zeros((ny, nx))
    for i in range(ny):
        twod[i] = normalized_intensities*-1+1.0


    for index in range(0,len(neon_wavelengths)):
        neon_w = neon_wavelengths[index]
        
        _xlim = [
            find_nearest_index(wavelengths,neon_w)-window_h,
            find_nearest_index(wavelengths,neon_w)+window_h,
        ]
        _ylim = [0,ny]
        _twod = twod[_ylim[0]:_ylim[1], _xlim[0]:_xlim[1]]
        wavelength_text = str(int(neon_w*100)/100)
        
        res = cv2.resize(np.uint8(_twod * 255), dsize=(IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_CUBIC)
        img = Image.fromarray(res)

        prob = random.random()
        
        if prob < TESTDATA_TRAINDATA_RATIO:
            p = os.path.join(TEST_DATA_PATH,'{:s}.{:06d}.BMP'.format(wavelength_text,filename_counter[index]))
        else:
            p = os.path.join(TRAIN_DATA_PATH,wavelength_text,'{:s}.{:06d}.BMP'.format(wavelength_text,filename_counter[index]))

        img.save(p, format='BMP')
        
        filename_counter[index] += 1


    

In [ ]:
from pathlib import Path


Goal: Avoid training with images having a very high similarity

Method: Naively, remove any images with a very high similarity 

In [ ]:
# with open('vergleich.txt', 'w') as vergleich_f:
#     train_path = Path(TRAIN_DATA_PATH)
#     train_files = list(train_path.glob('**/*.BMP'))
#     train_files_l = len(train_files)
#     part = train_files_l * PERCENT/ 100
#     now0 = datetime.now()
#     report = []
#     i = 0
#     j = 0
#     while len(train_files) > 0:
#         if int(i%part)==0:
#             print("{}: {} completed".format(datetime.now(),int(i/part*PERCENT)))
     
#         l_file = train_files.pop()
#         j = 0
#         for r_file in train_files:
#             j += 1
# #            if l_file.parts[-2] != r_file.parts[-2]:
# #                pass
# #                if l_file.exists() and r_file.exists():
# #                    pass
#                     #l_img = load_image(str(l_file))
#                     #r_img = load_image(str(r_file))
#                     #ssim =tf.image.ssim(l_img, r_img, max_val=255, filter_size=11, filter_sigma=1.5, k1=0.01, k2=0.03)


#             with Image.open(str(l_file)) as im1:
#                 bytes1 = im1.tobytes()
#             #ref_rec = list(bytes1)[0:31]

#             with Image.open(str(r_file)) as im2:
#                 bytes2 = im2.tobytes()
#             #inp_rec = list(bytes2)[0:31]

    #         #ref_time = np.correlate(ref_rec,ref_rec)
    #         #inp_time = np.correlate(ref_rec,inp_rec)
    #         #diff_time = abs(ref_time-inp_time)
                    
    #         diff_time = abs( np.correlate(list(bytes1)[0:31],list(bytes1)[0:31]) - np.correlate(list(bytes1)[0:31],list(bytes2)[0:31]))

    #                 #vergleich_f.write("{} {} {} \n".format(
    #                 #        l_file.parts[-2:], 
    #                 #        r_file.parts[-2:], 
    #                 #        diff_time
    #                 #        )
    #                 #)
    #                 #if ssim > MAX_SSIM:
    #                 #    l_file.unlink()
    #                 #    r_file.unlink()
    #         report.append({'l_file':l_file.parts[-1], 'r_file': r_file.parts[-1], 'ssim': diff_time})


    #     i += 1
    # now1 = datetime.now()
    # print(now1 - now0)
    

In [ ]:
#print(len(report))

In [ ]:
#with open('report.txt','w') as report_out:
#    for r in report:
#        report_out.write(str(r))


In [ ]:
train_images, train_labels  = load_images(TRAIN_DATA_PATH)


train_images, train_labels  = shuffle(train_images,train_labels,random_state=21)
print (len(train_labels))
xTrain, xTest, yTrain, yTest = train_test_split(train_images, train_labels, test_size = 0.2, random_state = 0)
optimizer = tf.keras.optimizers.legacy.Adam()

for i in range(6,7):
    DROPOUT = 0.1 * i
    model = build_model()
    #model.summary()
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics = ["accuracy"])
    tf_callback = tf.keras.callbacks.TensorBoard(log_dir="logs",histogram_freq=5)
    model.fit(xTrain,yTrain, epochs=NUM_EPOCHS,batch_size=NUM_BATCHES, callbacks=[tf_callback], verbose=1,validation_split=0.2)

    results = model.evaluate(xTest,yTest,verbose=1)
    print("--- Ergebnisse {} ----".format(TRAIN_DATA_PATH))
    print('Evaluation / Loss {}, Acc:{}'.format(results[0],results[1]))

    labels_decoded = []
    for directory in sorted(os.listdir(TRAIN_DATA_PATH)):
        if(directory.startswith('.') == False):
            labels_decoded.append(directory)

    corrects, wrongs = test_dataset(model, labels_decoded, TEST_DATA_PATH, resultfile='results_DROPOUT_{:04.2f}.txt'.format(DROPOUT))
    print("DROPOUT:", DROPOUT, corrects, wrongs)